## Setup

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import pandas as pd
import numpy as np
import os

# To make this notebook's output stable across runs
np.random.seed(42)

# To plot the figures
import matplotlib as mlp 
import matplotlib.pyplot as plt
mlp.rc('axes', labelsize=14)
mlp.rc('xtick', labelsize=12)
mlp.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "../"
IMAGES_PATH = "images"
os.makedirs(IMAGES_PATH, exist_ok=True)

# Function to save the figures
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300, figure=None):
    path = os.path.join(IMAGES_PATH, (fig_id + "." + fig_extension))
    print('Saving figure', fig_id)
    if tight_layout:
        plt.tight_layout()

    if figure is None:
        plt.savefig(path, format=fig_extension, dpi=resolution)
    else:
        figure.savefig(path, format=fig_extension, dpi=resolution)


## Get the  data

In [5]:
DATASETS_PATH = os.path.join(PROJECT_ROOT_DIR, 'datasets')
os.makedirs(DATASETS_PATH, exist_ok=True)

DATASET_NAME = 'dados 2-4.txt'

# columns names for the dataset
columns=['Distance','RSSI']

def load_data(data_path = DATASETS_PATH, dataset_name= DATASET_NAME, columns_names=columns):
    txt_path = os.path.join(data_path, dataset_name)
    return pd.read_table(txt_path, names=columns)

# read the dataset
data_2_4_Ghz = load_data(data_path=DATASETS_PATH, dataset_name=DATASET_NAME, columns_names=columns)
data_2_4_Ghz.head()

,Distance,RSSI
0,1,-21.0
1,2,-30.0
2,3,-29.0
3,4,-33.0
4,5,-32.0


In [9]:
# Split data in samples and labels

X = data_2_4_Ghz.drop(['RSSI'], axis=1)
y = data_2_4_Ghz['RSSI']

## Load Random Forest Model

In [3]:
import joblib

forest_reg_best = joblib.load('../2-4 GHz Analysis/models/forest_reg_best.pkl')

In [10]:
# Random forest predict

forest_prediction = forest_reg_best.predict(X)
print(forest_prediction)

[-28.3972995  -28.3972995  -29.38352757 -30.72488722 -32.87131266
 -37.33063127 -39.83672776 -39.66721021 -40.0520849  -41.26054781
 -41.39567588 -41.09657343 -41.90161642 -44.18605514 -44.36066228
 -44.76893709 -45.3020745  -46.77870345 -47.3574245  -47.66031322
 -47.74819091 -47.21540363 -47.25871416 -47.08318835 -45.87075677
 -45.41785464 -45.22864361 -45.21405589 -45.25864536 -45.25864536]


### Save the datas

In [34]:
# Save the datas in a dataframe to visualize the predictions by distance

df_visualization_forest_prediction = pd.DataFrame()
df_visualization_forest_prediction['Distance'] = X
df_visualization_forest_prediction['RSSI_Forest'] = forest_prediction
df_visualization_forest_prediction['RSSI_real'] = y

In [64]:
# Duplicate the samples in the reverse order
df_visualization_forest_prediction_reverse = pd.DataFrame()

# Creates a dataframe with the samples in the reverse order
df_visualization_forest_prediction_reverse = pd.DataFrame({
    'Distance': X.values.flatten()[::-1],
    'RSSI_Forest': forest_prediction[::-1],
    'RSSI_real': y.values[::-1]
})


In [71]:
# Joins the datas

df_visualization = pd.concat([df_visualization_forest_prediction, df_visualization_forest_prediction_reverse], ignore_index=True)


In [74]:
# Add column that show average RSSI predict for each five samples

df_visualization['RSSI_Forest_Mean'] = df_visualization['RSSI_Forest'].rolling(window=5, min_periods=1).mean()

In [75]:
# Add column that show the maximum and minimum RSSI predict

df_visualization['RSSI_Forest_Max'] = df_visualization['RSSI_Forest'].cummax()
df_visualization['RSSI_Forest_Min'] = df_visualization['RSSI_Forest'].cummin()


In [76]:
df_visualization

,Distance,RSSI_Forest,RSSI_real,RSSI_Forest_Mean,RSSI_Forest_Max,RSSI_Forest_Min
0,1,-28.397299,-21.00,-28.397299,-28.397299,-28.397299
1,2,-28.397299,-30.00,-28.397299,-28.397299,-28.397299
2,3,-29.383528,-29.00,-28.726042,-28.397299,-29.383528
3,4,-30.724887,-33.00,-29.225753,-28.397299,-30.724887
4,5,-32.871313,-32.00,-29.954865,-28.397299,-32.871313
5,6,-37.330631,-38.50,-31.741532,-28.397299,-37.330631
6,7,-39.836728,-43.25,-34.029417,-28.397299,-39.836728
7,8,-39.667210,-37.50,-36.086154,-28.397299,-39.836728
8,9,-40.052085,-41.00,-37.951593,-28.397299,-40.052085
9,10,-41.260548,-42.50,-39.629440,-28.397299,-41.260548
